In [205]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
#export
from fastml.core import *
from fastml.data.datasets import *
from fastml.model import loss as loss
from fastml.model import metrics as metrics
from fastml.model import optimizers
from torch import nn

In [147]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = loss.mse
        
    def __call__(self, x, targ):
        for layer in self.layers: x = layer(x)
        return self.loss(x.squeeze(), targ)

In [148]:
train_ds,valid_ds = Datasets.MNIST()

In [149]:
train_ds.x.shape

torch.Size([50000, 784])

In [150]:
model = Model(784, 50, 1)

## Training

In [151]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)

    def __call__(self, x): return self.l2(F.relu(self.l1(x)))

In [152]:
for name,l in model.named_children(): print(f"{name}: {l}")

In [153]:
model

Model()

In [154]:
bs=64 # batch size

model = Model(784, 50, 10)
xb, yb = train_ds[0:bs] # a mini-batch from x
preds = model(xb)      # predictions

preds[0], preds.shape

(tensor([ 0.1501, -0.2373, -0.0210, -0.2173, -0.1749, -0.1409, -0.1625,  0.1527,
         -0.0293,  0.0479], grad_fn=<SelectBackward>), torch.Size([64, 10]))

In [155]:
accuracy = metrics.accuracy(preds, yb[0:bs])

## basic train

In [181]:
lr = 0.5
epochs = 2
batch_size=64
loss_func = loss.cross_entropy()
model = Model(784, 50, 10)

def fit():
    for epoch in range(epochs):
        for batch in range(x_train.shape[0]//batch_size):
            
            b_from = batch*batch_size
            b_to = b_from + batch_size
            x_batch, y_batch = train_ds[b_from:b_to]
            
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= lr * p.grad
                model.zero_grad()
            
        accuracy = metrics.accuracy(model(x_train), y_train)
        print("epoch %s accuracy is %f3" % (epoch, accuracy.item()*100))

In [173]:
fit()
assert metrics.accuracy(model(x_train), y_train).item()*100 > 80

epoch 0 accuracy is 91.0179973
epoch 1 accuracy is 90.3980023


In [174]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [175]:
layers = [nn.Linear(784,50), nn.ReLU(), nn.Linear(50,10)]
model = SequentialModel(layers)

fit()
assert metrics.accuracy(model(x_train), y_train).item()*100 > 80

epoch 0 accuracy is 91.2240033
epoch 1 accuracy is 92.2980013


In [176]:
model = nn.Sequential(nn.Linear(784,50), nn.ReLU(), nn.Linear(50,10))

fit()
assert metrics.accuracy(model(x_train), y_train).item()*100 > 80

epoch 0 accuracy is 85.9399973
epoch 1 accuracy is 90.9940003


In [161]:
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

In [162]:
def create_model():
    return nn.Sequential(nn.Linear(784,50), nn.ReLU(), nn.Linear(50,10))

In [182]:
model = create_model()

fit()
assert metrics.accuracy(model(x_train), y_train).item()*100 > 80

epoch 0 accuracy is 88.8140023
epoch 1 accuracy is 91.9340013


## with optimizer

In [216]:
model = create_model()
optimizer = optimizers.SGD(model)

In [217]:
def fit():
    for epoch in range(epochs):
        for batch in range(x_train.shape[0]//batch_size):
            
            b_from = batch*batch_size
            b_to = b_from + batch_size
            x_batch, y_batch = train_ds[b_from:b_to]
            
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        accuracy = metrics.accuracy(model(x_train), y_train)
        print("epoch %s accuracy is %f3" % (epoch, accuracy.item()*100))

In [218]:
fit()
assert metrics.accuracy(model(x_train), y_train).item()*100 > 80

epoch 0 accuracy is 90.9699983
epoch 1 accuracy is 92.1000003


## with data loader

In [222]:
train_dl,_ = wih_data_loader(train_ds, valid_ds, batch_size)

In [223]:
model = create_model()
optimizer = optimizers.SGD(model)

def fit():
    for epoch in range(epochs):
        for x_batch, y_batch in train_dl:
            
            loss = loss_func(model(x_batch), y_batch)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        accuracy = metrics.accuracy(model(x_train), y_train)
        print("epoch %s accuracy is %f3" % (epoch, accuracy.item()*100))

In [224]:
fit()
assert metrics.accuracy(model(x_train), y_train).item()*100 > 80

epoch 0 accuracy is 90.1380003
epoch 1 accuracy is 92.8319993


## Export

In [83]:
!python notebook2script.py model_dev.ipynb fastml/model.py

Converted model_dev.ipynb to fastml/model.py
